In [4]:
import re
import pandas as pd
#from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
short = pd.read_csv('../intermediate-charge-density22.csv')
short['Class'] = 1

pos = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]
pos


,Sequence,Class
0,FIKHFIHRF,1
1,LIKHILHRL,1
2,LQLLKQLLKLLKQF,1
3,IKQLLHFFQRF,1
4,LKQFVHFIHRF,1
...,...,...
3045,GLLSVLKGVLKTAGKHIFKNVGGSLLDQAKCKISGEC,1
3046,GRDYRTSLTIVQKLKKMVD,1
3047,KRFKQDGGWSHWSPWSS,1
3048,PDEDAINNALNKVCSTGRRQRSICKQLLKK,1


In [5]:
import re
import pandas as pd
#from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
short = pd.read_csv('../neg-intermediate-charge-density22.csv')
short['Class'] = 0
neg = short[
    (short['Sequence'].str.len() > 6) &
    (short['Sequence'].str.len() < 40)
]
neg
# Display the result

neg = neg.sample(n=min(3050, len(neg)), random_state=40)
neg

,Sequence,Class
12093,IDVEKKKQKDVKFVHRMPISFL,0
1891,AKVDVPKRYIGWRAYVIVVRD,0
13051,CAIRGSALLVIVCTLIGISCYRMRELSK,0
24727,EQVLLLRLFGPFATACELLHNAKRSPRFR,0
23507,HYNKYLTRKRRLEIAHTLTLTERQI,0
...,...,...
6414,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0
8572,YNVHKHTDGKC,0
21720,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0
18632,QTLKLICKECGYTIHREGIRLRKLEIVG,0


In [6]:
concatenated_df = pd.concat([pos,neg], ignore_index=True)
concatenated_df = concatenated_df.drop_duplicates(subset='Sequence', keep=False)
concatenated_df

,Sequence,Class
0,FIKHFIHRF,1
1,LIKHILHRL,1
2,LQLLKQLLKLLKQF,1
3,IKQLLHFFQRF,1
4,LKQFVHFIHRF,1
...,...,...
6095,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0
6096,YNVHKHTDGKC,0
6097,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0
6098,QTLKLICKECGYTIHREGIRLRKLEIVG,0


In [7]:
import re
# Clean comma-separated sequences (take the first part)
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.split(',').str[0]

# Remove spaces
concatenated_df['Sequence'] = concatenated_df['Sequence'].str.replace(' ', '', regex=False)

# Remove invalid amino acids
concatenated_df['Sequence'] = concatenated_df['Sequence'].apply(lambda x: re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', x))

#from sklearn.model_selection import train_test_split

# Remove sequences with issues
concatenated_df =concatenated_df[~concatenated_df['Sequence'].str.contains('[, ZBOUXJ ]')]
# Optional: show progress bar
concatenated_df

,Sequence,Class
0,FIKHFIHRF,1
1,LIKHILHRL,1
2,LQLLKQLLKLLKQF,1
3,IKQLLHFFQRF,1
4,LKQFVHFIHRF,1
...,...,...
6095,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0
6096,YNVHKHTDGKC,0
6097,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0
6098,QTLKLICKECGYTIHREGIRLRKLEIVG,0


In [8]:
concatenated_df = concatenated_df.reset_index(drop=True)
concatenated_df

,Sequence,Class
0,FIKHFIHRF,1
1,LIKHILHRL,1
2,LQLLKQLLKLLKQF,1
3,IKQLLHFFQRF,1
4,LKQFVHFIHRF,1
...,...,...
6093,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0
6094,YNVHKHTDGKC,0
6095,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0
6096,QTLKLICKECGYTIHREGIRLRKLEIVG,0


In [ ]:
import requests
import os
import time
import pandas as pd

# Configuration
OUTPUT_DIR = "id/"
SLEEP_TIME = 5  # Seconds between requests
MAX_RETRIES = 3  # Retry attempts for failed requests
START_INDEX = 0  # <-- Change here to resume

# Create the output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_pdb_from_sequence(seq, out_file_path):
    """Get PDB structure from ESM Atlas API for a given amino acid sequence"""
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    headers = {
        "Content-Type": "text/plain"
    }

    # Sanitize sequence
    clean_seq = ''.join(filter(str.isalpha, seq)).upper()

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(url, data=clean_seq, headers=headers, timeout=30)
            if response.ok:
                with open(out_file_path, "w") as f:
                    f.write(response.text)
                return True
            else:
                print(f"Attempt {attempt + 1} failed for {clean_seq[:10]}... "
                      f"Status: {response.status_code}, Response: {response.text[:100]}")
                if response.status_code == 429:
                    time.sleep(SLEEP_TIME * (attempt + 2))  # Exponential backoff
        except Exception as e:
            print(f"Attempt {attempt + 1} error: {str(e)}")
            time.sleep(SLEEP_TIME)

    return False

# Initialize PDB_File column if missing
if 'PDB_File' not in concatenated_df.columns:
    concatenated_df['PDB_File'] = [None] * len(concatenated_df)

# Process each sequence from START_INDEX onward
for idx in range(START_INDEX, len(concatenated_df)):
    seq = concatenated_df.at[idx, 'Sequence']
    filename = f"peptide_{idx}.pdb"
    file_path = os.path.join(OUTPUT_DIR, filename)

    print(f"Processing sequence {idx + 1}/{len(concatenated_df)}: {seq[:15]}...")

    # Skip if already exists (optional)
    if os.path.exists(file_path):
        print(f"Already exists: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
        continue

    success = get_pdb_from_sequence(seq, file_path)
    if success:
        print(f"Successfully saved: {file_path}")
        concatenated_df.at[idx, 'PDB_File'] = file_path
    else:
        print(f"Failed to get PDB for index {idx}")
        concatenated_df.at[idx, 'PDB_File'] = None

    time.sleep(SLEEP_TIME)




In [9]:
import requests
import os
import time
import pandas as pd


# Configuration
OUTPUT_DIR = "../id/"
SLEEP_TIME = 5        # Seconds between requests
MAX_RETRIES = 3       # Retry attempts for failed requests
START_INDEX = 1450       # <-- Change this to resume from a specific index

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_pdb_from_sequence(seq, out_file_path):
    """Get PDB structure from ESM Atlas API for a given amino acid sequence"""
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    headers = {
        "Content-Type": "text/plain"
    }

    # Sanitize sequence
    clean_seq = ''.join(filter(str.isalpha, seq)).upper()

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(url, data=clean_seq, headers=headers, timeout=30)
            if response.ok:
                with open(out_file_path, "w") as f:
                    f.write(response.text)
                return True
            else:
                print(f"Attempt {attempt + 1} failed for {clean_seq[:10]}... "
                      f"Status: {response.status_code}, Response: {response.text[:100]}")
                if response.status_code == 429:
                    time.sleep(SLEEP_TIME * (attempt + 2))  # Exponential backoff
        except Exception as e:
            print(f"Attempt {attempt + 1} error: {str(e)}")
            time.sleep(SLEEP_TIME)

    return False

# Initialize PDB_File column if missing
if 'PDB_File' not in concatenated_df.columns:
    concatenated_df['PDB_File'] = [None] * len(concatenated_df)

# Process each sequence from START_INDEX onward
for pos in range(START_INDEX, len(concatenated_df)):
    seq = concatenated_df.iloc[pos]['Sequence']
    filename = f"peptide_{pos}.pdb"
    file_path = os.path.join(OUTPUT_DIR, filename)

    print(f"Processing sequence {pos + 1}/{len(concatenated_df)}: {seq[:15]}...")

    # Skip if file already exists
    if os.path.exists(file_path):
        print(f"Already exists: {file_path}")
        concatenated_df.iloc[pos, concatenated_df.columns.get_loc('PDB_File')] = file_path
        continue

    # Try to get PDB from sequence
    success = get_pdb_from_sequence(seq, file_path)
    if success:
        print(f"✅ Successfully saved: {file_path}")
        concatenated_df.iloc[pos, concatenated_df.columns.get_loc('PDB_File')] = file_path
    else:
        print(f"❌ Failed to get PDB for index {pos}")
        concatenated_df.iloc[pos, concatenated_df.columns.get_loc('PDB_File')] = None

    time.sleep(SLEEP_TIME)



Processing sequence 1451/6098: QLKVDLWGTRSGIQP...
Already exists: ../id/peptide_1450.pdb
Processing sequence 1452/6098: MAFKKLEKVGRNIRD...
Already exists: ../id/peptide_1451.pdb
Processing sequence 1453/6098: GFFALIPKIISSPLF...
Already exists: ../id/peptide_1452.pdb
Processing sequence 1454/6098: GFFALIAKIISSPLF...
Already exists: ../id/peptide_1453.pdb
Processing sequence 1455/6098: RGGRLCYCRRRFCVC...
Already exists: ../id/peptide_1454.pdb
Processing sequence 1456/6098: FWGALAKGALKLIGV...
Already exists: ../id/peptide_1455.pdb
Processing sequence 1457/6098: SFFKKAAHVGKHVGK...
Already exists: ../id/peptide_1456.pdb
Processing sequence 1458/6098: KKAAHVGKHVGKAAL...
Already exists: ../id/peptide_1457.pdb
Processing sequence 1459/6098: GWGSFFKKAAHVGKH...
Already exists: ../id/peptide_1458.pdb
Processing sequence 1460/6098: RWKIFKKIEKVGRNV...
Already exists: ../id/peptide_1459.pdb
Processing sequence 1461/6098: GMWSKIKNAGKAAKA...
Already exists: ../id/peptide_1460.pdb
Processing sequence 1

Already exists: ../id/peptide_2109.pdb
Processing sequence 2111/6098: GLKRIFKSGLGKLVK...
Already exists: ../id/peptide_2110.pdb
Processing sequence 2112/6098: ASWKVFLKNIGKAAG...
Already exists: ../id/peptide_2111.pdb
Processing sequence 2113/6098: RQYMRQIEQALRYGY...
Already exists: ../id/peptide_2112.pdb
Processing sequence 2114/6098: RDNIQGITKPAIRRL...
Already exists: ../id/peptide_2113.pdb
Processing sequence 2115/6098: VDKPPYLPRPPPPRR...
Already exists: ../id/peptide_2114.pdb
Processing sequence 2116/6098: VDKPPYLPRPTPPRR...
Already exists: ../id/peptide_2115.pdb
Processing sequence 2117/6098: VDKPPYLPRPHPPRR...
Already exists: ../id/peptide_2116.pdb
Processing sequence 2118/6098: VDKPPYLPRPKPPRR...
Already exists: ../id/peptide_2117.pdb
Processing sequence 2119/6098: WDPYFAGVKKLTKAI...
Already exists: ../id/peptide_2118.pdb
Processing sequence 2120/6098: WGKFFAGVKKLTKAI...
Already exists: ../id/peptide_2119.pdb
Processing sequence 2121/6098: FFGHLFKLATKIIPS...
Already exists: ../id

Already exists: ../id/peptide_2653.pdb
Processing sequence 2655/6098: SIGTAVKKAVPIAKK...
Already exists: ../id/peptide_2654.pdb
Processing sequence 2656/6098: SIWGDIGQGVGKAAY...
Already exists: ../id/peptide_2655.pdb
Processing sequence 2657/6098: SLFGTFAKMALKGAS...
Already exists: ../id/peptide_2656.pdb
Processing sequence 2658/6098: SLGPAIKATRQVCPK...
Already exists: ../id/peptide_2657.pdb
Processing sequence 2659/6098: SPAGCRFCCGCCPNM...
Already exists: ../id/peptide_2658.pdb
Processing sequence 2660/6098: SVIGCWTKSIPPRPC...
Already exists: ../id/peptide_2659.pdb
Processing sequence 2661/6098: SWLDRIWRWICKVLS...
Already exists: ../id/peptide_2660.pdb
Processing sequence 2662/6098: SWLRDIWRWICKVLS...
Already exists: ../id/peptide_2661.pdb
Processing sequence 2663/6098: TWLPAVIKIQAHWRG...
Already exists: ../id/peptide_2662.pdb
Processing sequence 2664/6098: VAPIAKYLATALAKW...
Already exists: ../id/peptide_2663.pdb
Processing sequence 2665/6098: VCSCRLVFCRRTELR...
Already exists: ../id

Already exists: ../id/peptide_3266.pdb
Processing sequence 3268/6098: VNGPRQWGPVLANSQ...
Already exists: ../id/peptide_3267.pdb
Processing sequence 3269/6098: ELKPIDTRYPRRRQA...
Already exists: ../id/peptide_3268.pdb
Processing sequence 3270/6098: GPLLAGIVFDNIGSK...
Already exists: ../id/peptide_3269.pdb
Processing sequence 3271/6098: AKNIQYFGAGTRLSV...
Already exists: ../id/peptide_3270.pdb
Processing sequence 3272/6098: PAERSRRSRNLWVGD...
Already exists: ../id/peptide_3271.pdb
Processing sequence 3273/6098: DKQKESLKSKWKLSG...
Already exists: ../id/peptide_3272.pdb
Processing sequence 3274/6098: RIVIKLLVTYLERNH...
Already exists: ../id/peptide_3273.pdb
Processing sequence 3275/6098: CTFGLRSCFHRKVSD...
Already exists: ../id/peptide_3274.pdb
Processing sequence 3276/6098: ESDVKTFNYTAHLWD...
Already exists: ../id/peptide_3275.pdb
Processing sequence 3277/6098: CMWGHADVISSNIAI...
Already exists: ../id/peptide_3276.pdb
Processing sequence 3278/6098: RYSLRNASSRRTVLL...
Already exists: ../id

Already exists: ../id/peptide_3944.pdb
Processing sequence 3946/6098: VNKGRKLPYFIHHFE...
Already exists: ../id/peptide_3945.pdb
Processing sequence 3947/6098: GTGKASSTATGTTKK...
Already exists: ../id/peptide_3946.pdb
Processing sequence 3948/6098: YIRILKGDKVTVALT...
Already exists: ../id/peptide_3947.pdb
Processing sequence 3949/6098: NLASDLIAYLFRKVR...
Already exists: ../id/peptide_3948.pdb
Processing sequence 3950/6098: PSDVIKGRLNFCFRR...
Already exists: ../id/peptide_3949.pdb
Processing sequence 3951/6098: KQGISGPL...
Already exists: ../id/peptide_3950.pdb
Processing sequence 3952/6098: LVKGKEAARACVEIL...
Already exists: ../id/peptide_3951.pdb
Processing sequence 3953/6098: VGEVFRVLHLSAPGR...
Already exists: ../id/peptide_3952.pdb
Processing sequence 3954/6098: VATSGGTSKKEYRKK...
Already exists: ../id/peptide_3953.pdb
Processing sequence 3955/6098: TPRKSAVSGTKISSD...
Already exists: ../id/peptide_3954.pdb
Processing sequence 3956/6098: LWTHAQVVGERYGSK...
Already exists: ../id/peptid

Already exists: ../id/peptide_4547.pdb
Processing sequence 4549/6098: KRLAPDRNTSDWKGK...
Already exists: ../id/peptide_4548.pdb
Processing sequence 4550/6098: VAPIPVKISIVQIRH...
Already exists: ../id/peptide_4549.pdb
Processing sequence 4551/6098: LRSRCPDAKVMKQSI...
Already exists: ../id/peptide_4550.pdb
Processing sequence 4552/6098: AKEKYGAYFARIVRS...
Already exists: ../id/peptide_4551.pdb
Processing sequence 4553/6098: KTPGSRRHYQFSNGV...
Already exists: ../id/peptide_4552.pdb
Processing sequence 4554/6098: YNHTRKSIRIGPGRA...
Already exists: ../id/peptide_4553.pdb
Processing sequence 4555/6098: AGAVLEDRWETRAQV...
Already exists: ../id/peptide_4554.pdb
Processing sequence 4556/6098: GGKYPVQTHGDLITR...
Already exists: ../id/peptide_4555.pdb
Processing sequence 4557/6098: IRRPILERGTKAVLK...
Already exists: ../id/peptide_4556.pdb
Processing sequence 4558/6098: DERLMRVHKRLTRLI...
Already exists: ../id/peptide_4557.pdb
Processing sequence 4559/6098: KTLSFSKKLANHIGA...
Already exists: ../id

Already exists: ../id/peptide_5277.pdb
Processing sequence 5279/6098: YLQKMHLMLSVSSVI...
Already exists: ../id/peptide_5278.pdb
Processing sequence 5280/6098: QRSLPSSQI...
Already exists: ../id/peptide_5279.pdb
Processing sequence 5281/6098: AKIQELSQHVPKGCL...
Already exists: ../id/peptide_5280.pdb
Processing sequence 5282/6098: AVYRIVAIDVRSRRE...
Already exists: ../id/peptide_5281.pdb
Processing sequence 5283/6098: VSKEIRRRRCVHVHD...
Already exists: ../id/peptide_5282.pdb
Processing sequence 5284/6098: NRLWRKTRKPYGAFC...
Already exists: ../id/peptide_5283.pdb
Processing sequence 5285/6098: PQRASEAGKKGGQNS...
Already exists: ../id/peptide_5284.pdb
Processing sequence 5286/6098: TCGAVRIWRCEKCRL...
Already exists: ../id/peptide_5285.pdb
Processing sequence 5287/6098: NKRADYIKNWWNIVN...
Already exists: ../id/peptide_5286.pdb
Processing sequence 5288/6098: RRWISAKWEQPPRDI...
Already exists: ../id/peptide_5287.pdb
Processing sequence 5289/6098: STRLAGGTAAHAVNG...
Already exists: ../id/pepti

Already exists: ../id/peptide_5981.pdb
Processing sequence 5983/6098: AKWAQKWLEPGKSFA...
Already exists: ../id/peptide_5982.pdb
Processing sequence 5984/6098: FIIKVKKILECICVN...
Already exists: ../id/peptide_5983.pdb
Processing sequence 5985/6098: FFSPRKERTQATSDK...
Already exists: ../id/peptide_5984.pdb
Processing sequence 5986/6098: NSGGSNAKLTFGKGT...
Already exists: ../id/peptide_5985.pdb
Processing sequence 5987/6098: VLLVCSALVMVVVVR...
Already exists: ../id/peptide_5986.pdb
Processing sequence 5988/6098: GRHIEKTSHSAGFFV...
Already exists: ../id/peptide_5987.pdb
Processing sequence 5989/6098: YIHYYNHHRLKLKLK...
Already exists: ../id/peptide_5988.pdb
Processing sequence 5990/6098: LRRDNGPQV...
Already exists: ../id/peptide_5989.pdb
Processing sequence 5991/6098: ARFGPRVQPAPRLQH...
Already exists: ../id/peptide_5990.pdb
Processing sequence 5992/6098: CGFHNSEIPKSISAF...
Already exists: ../id/peptide_5991.pdb
Processing sequence 5993/6098: LLLLNASFLRGRRVS...
Already exists: ../id/pepti

In [10]:
import os

OUTPUT_DIR = "../id/"
existing_files = os.listdir(OUTPUT_DIR)

for file in existing_files:
    if file.endswith(".pdb") and file.startswith("peptide_"):
        try:
            idx = int(file.replace("peptide_", "").replace(".pdb", ""))
            file_path = os.path.join(OUTPUT_DIR, file)
            if 0 <= idx < len(concatenated_df):
                concatenated_df.at[idx, 'PDB_File'] = file_path
        except ValueError:
            continue  # In case there's a filename that doesn't match the pattern


concatenated_df.to_csv("peptides_with_pdb_paths_fixed.csv", index=False)
print("✅ PDB_File column updated from existing files.")
concatenated_df

✅ PDB_File column updated from existing files.


,Sequence,Class,PDB_File
0,FIKHFIHRF,1,../id/peptide_0.pdb
1,LIKHILHRL,1,../id/peptide_1.pdb
2,LQLLKQLLKLLKQF,1,../id/peptide_2.pdb
3,IKQLLHFFQRF,1,../id/peptide_3.pdb
4,LKQFVHFIHRF,1,../id/peptide_4.pdb
...,...,...,...
6093,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0,../id/peptide_6093.pdb
6094,YNVHKHTDGKC,0,../id/peptide_6094.pdb
6095,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0,../id/peptide_6095.pdb
6096,QTLKLICKECGYTIHREGIRLRKLEIVG,0,../id/peptide_6096.pdb


In [11]:
from Bio.PDB import PDBParser, DSSP
import pandas as pd
import numpy as np
import os

# Path to folder containing PDB files
pdb_directory = "../id/" 

# Initialize DSSP feature columns
dssp_columns = ['Mean_RSA', 'Mean_Phi', 'Mean_Psi']
for col in dssp_columns:
    concatenated_df[col] = np.nan

parser = PDBParser(QUIET=True)

# Loop to calculate RSA, Phi, Psi features
for idx in concatenated_df.index:
    pdb_filename = f"peptide_{idx}.pdb"
    pdb_path = os.path.join(pdb_directory, pdb_filename)

    if not os.path.exists(pdb_path):
        print(f"File not found: {pdb_path}")
        continue

    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_path)
        model = structure[0]
        dssp = DSSP(model, pdb_path)

        rsa_list = []
        phi_list = []
        psi_list = []

        for res in dssp:
            rsa = res[3]
            phi = res[4]
            psi = res[5]

            rsa_list.append(rsa)
            phi_list.append(phi)
            psi_list.append(psi)

        # Compute means
        concatenated_df.at[idx, 'Mean_RSA'] = np.mean(rsa_list) if rsa_list else np.nan
        concatenated_df.at[idx, 'Mean_Phi'] = np.mean(phi_list) if phi_list else np.nan
        concatenated_df.at[idx, 'Mean_Psi'] = np.mean(psi_list) if psi_list else np.nan

    except Exception as e:
        print(f"Error processing index {idx} - file: {pdb_path}: {e}")


In [12]:
from Bio.PDB import PDBParser, DSSP
import os

# Columns to store DSSP secondary structure percentages
helices = []
sheets = []
coils = []

parser = PDBParser()

for idx in concatenated_df.index:
    pdb_file = f"../id//peptide_{idx}.pdb"
    try:
        structure = parser.get_structure(f"pep_{idx}", pdb_file)
        model = structure[0]
        dssp = DSSP(model, pdb_file)
        
        ss_list = [dssp[key][2] for key in dssp.keys()]
        total = len(ss_list)
        
        helix_count = sum(s in "HGI" for s in ss_list)
        sheet_count = sum(s in "BE" for s in ss_list)
        coil_count  = sum(s in "ST " for s in ss_list)  # includes turn, bend, and loops
        
        helices.append(helix_count / total * 100)
        sheets.append(sheet_count / total * 100)
        coils.append(coil_count / total * 100)
    except Exception as e:
        print(f"Failed to process {pdb_file}: {e}")
        helices.append(None)
        sheets.append(None)
        coils.append(None)

# Add DSSP % columns to DataFrame
concatenated_df['%Helix'] = helices
concatenated_df['%Sheet'] = sheets
concatenated_df['%Coil'] = coils


In [13]:
import mdtraj as md
import pandas as pd
import numpy as np
import os

# Folder containing your PDBs
pdb_folder = '../id/'

# Initialize columns
concatenated_df['RoG'] = np.nan
concatenated_df['SASA'] = np.nan
concatenated_df['Compactness'] = np.nan

# Loop through each peptide
for idx in concatenated_df.index:
    pdb_path = os.path.join(pdb_folder, f"peptide_{idx}.pdb")

    if not os.path.exists(pdb_path):
        print(f"Missing: {pdb_path}")
        continue

    try:
        traj = md.load(pdb_path)
        
        # Radius of Gyration (RoG)
        rog = md.compute_rg(traj)[0]
        concatenated_df.at[idx, 'RoG'] = rog

        # Solvent Accessible Surface Area (SASA)
        sasa = md.shrake_rupley(traj)[0].sum()
        concatenated_df.at[idx, 'SASA'] = sasa

        # Compactness = Mass / SASA (or optionally 1/SASA per atom)
        n_atoms = traj.n_atoms
        compactness = n_atoms / sasa if sasa > 0 else np.nan
        concatenated_df.at[idx, 'Compactness'] = compactness

    except Exception as e:
        print(f"Error at index {idx}: {e}")


/home/hrushikesh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
cleaned_df = concatenated_df
cleaned_df

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness
0,FIKHFIHRF,1,../id/peptide_0.pdb,0.895877,-36.344444,69.555556,0.000000,0.000000,44.444444,0.775664,15.983317,5.568306
1,LIKHILHRL,1,../id/peptide_1.pdb,0.924433,-0.277778,105.388889,0.000000,0.000000,0.000000,0.826213,15.638212,5.115674
2,LQLLKQLLKLLKQF,1,../id/peptide_2.pdb,0.703186,-36.621429,-9.914286,85.714286,0.000000,0.000000,0.719872,17.867218,6.772179
3,IKQLLHFFQRF,1,../id/peptide_3.pdb,0.753307,-32.572727,0.772727,81.818182,0.000000,0.000000,0.642931,15.621991,6.721294
4,LKQFVHFIHRF,1,../id/peptide_4.pdb,0.768256,-34.781818,3.081818,81.818182,0.000000,0.000000,0.642681,15.822387,6.636167
...,...,...,...,...,...,...,...,...,...,...,...,...
6093,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0,../id/peptide_6093.pdb,0.640253,-48.727586,-28.479310,93.103448,0.000000,0.000000,1.300180,29.070087,7.292720
6094,YNVHKHTDGKC,0,../id/peptide_6094.pdb,0.806652,-17.345455,99.027273,0.000000,0.000000,27.272727,0.736583,14.561454,6.180702
6095,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0,../id/peptide_6095.pdb,0.642829,-48.340000,17.716667,56.666667,0.000000,13.333333,1.144406,35.701397,7.254618
6096,QTLKLICKECGYTIHREGIRLRKLEIVG,0,../id/peptide_6096.pdb,0.782445,-49.221429,95.907143,0.000000,28.571429,10.714286,1.473323,37.902214,5.989096


In [15]:
import pandas as pd
import peptides



# Define the peptide descriptors you want to calculate
descriptors = [
    'frequencies',
    'aliphatic_index',
    'boman',
    'charge',
    'isoelectric_point',
    'hydrophobic_moment',
    'hydrophobicity',
    'instability_index',
    'mass_shift',
    'molecular_weight',
    'mz',
    'structural_class'
]

# Create a dictionary to store descriptor values
descriptor_values_dict = {}

# Iterate through each descriptor and calculate values for each sequence
for descriptor in descriptors:
    descriptor_values = []
    for sequence in cleaned_df['Sequence']:
        peptide = peptides.Peptide(sequence)
        value = getattr(peptide, descriptor)()
        descriptor_values.append(value)
    descriptor_values_dict[descriptor] = descriptor_values

# Create a new DataFrame using the descriptor values dictionary
descriptors = pd.DataFrame(descriptor_values_dict)

In [16]:
import pandas as pd


# containing dictionaries of amino acid frequencies

# Expand 'frequencies' column into separate columns
amino_acid_frequencies = pd.DataFrame(descriptors['frequencies'].tolist()).fillna(0)

# Combine the expanded DataFrame with the original DataFrame
descriptors_df = pd.concat([descriptors, amino_acid_frequencies], axis=1)

# Drop the original 'frequencies' column
descriptors_df.drop(columns=['frequencies'], inplace=True)

# Display the resulting DataFrame
descriptors_df


,aliphatic_index,boman,charge,isoelectric_point,hydrophobic_moment,hydrophobicity,instability_index,mass_shift,molecular_weight,mz,...,T,W,Y,V,O,U,B,Z,J,X
0,86.666667,1.223333,2.179504,11.651804,0.694915,0.288889,1.644444,12.040258,1244.50764,622.856176,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,216.666667,0.576667,2.179504,11.651804,0.672117,0.622222,69.677778,12.040258,1142.45604,571.879651,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,195.000000,-0.296429,2.997099,11.103071,0.784091,0.514286,36.757143,18.060387,1726.22204,863.571281,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,106.363636,1.137273,2.088595,11.651790,0.803520,0.172727,13.472727,12.040258,1476.78734,738.927331,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,97.272727,1.137273,2.179504,11.651804,0.790389,0.236364,17.836364,12.040258,1471.77094,736.419671,...,0.000000,0.000000,0.000000,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,77.586207,1.873103,4.998864,12.262729,0.691267,-0.396552,23.582759,36.120774,3045.55634,1522.848516,...,0.034483,0.000000,0.000000,0.034483,0.0,0.0,0.0,0.0,0.0,0.0
6094,26.363636,2.930000,1.116834,8.533599,0.154657,-1.536364,16.045455,12.040258,1301.44244,679.817128,...,0.090909,0.000000,0.090909,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
6095,126.666667,0.551000,3.932349,9.820105,0.615387,0.103333,25.626667,30.100645,3699.51104,1878.037543,...,0.033333,0.000000,0.166667,0.033333,0.0,0.0,0.0,0.0,0.0,0.0
6096,121.785714,1.733571,2.968524,9.459829,0.625411,-0.139286,19.242857,36.120774,3270.94564,1692.441925,...,0.071429,0.000000,0.035714,0.035714,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem


# Create a new DataFrame to store the results
d4 = pd.DataFrame()

def calculate_tpsa(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate TPSA using AllChem method
        tpsa = AllChem.CalcTPSA(peptide_molecule)

        return tpsa
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create a new 'TPSA' column
#new_pos['Sequence'] = cleaned_df['Sequence']
d4['TPSA'] = cleaned_df['Sequence'].apply(calculate_tpsa)

# Display the new DataFrame with TPSA values
d4

,TPSA
0,441.40
1,441.40
2,648.95
3,557.10
4,542.69
...,...
6093,1369.14
6094,584.57
6095,1369.96
6096,1357.14


In [18]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Crippen, Lipinski, Descriptors


# Create a new DataFrame to store the results
d6 = pd.DataFrame()

def calculate_descriptors(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate various molecular descriptors
        heavy_atom_count = Descriptors.HeavyAtomCount(peptide_molecule)
        logp = Crippen.MolLogP(peptide_molecule)
        mol_logp = Crippen.MolLogP(peptide_molecule)
        # MolLogS calculation is not directly available in RDKit, consider using other methods/tools
        fraction_csp3 = Lipinski.FractionCSP3(peptide_molecule)
        bertz_ct = Descriptors.BertzCT(peptide_molecule)

        return heavy_atom_count, logp, mol_logp, fraction_csp3, bertz_ct
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d6[['HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT']] = cleaned_df['Sequence'].apply(calculate_descriptors).apply(pd.Series)

# Display the new DataFrame with descriptor values
d6

,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT
0,90.0,0.42087,0.42087,0.460317,3067.086602
1,81.0,-0.16893,-0.16893,0.703704,2297.411226
2,122.0,-0.55040,-0.55040,0.726190,3511.063001
3,106.0,-0.99823,-0.99823,0.520548,3515.886152
4,106.0,-0.68803,-0.68803,0.493151,3644.669989
...,...,...,...,...,...
6093,213.0,-17.14109,-17.14109,0.676923,7002.208271
6094,91.0,-6.66770,-6.66770,0.545455,2919.480801
6095,260.0,-5.00823,-5.00823,0.634286,9329.927351
6096,228.0,-10.02109,-10.02109,0.694444,7619.734557


In [19]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors


# Create a new DataFrame to store the results
d7 = pd.DataFrame()

def calculate_polarity(peptide_sequence):
    # Create a molecule object from the peptide sequence
    peptide_molecule = Chem.MolFromSequence(peptide_sequence)

    if peptide_molecule is not None:
        # Calculate descriptors related to polarity
        num_h_acceptors = Lipinski.NumHAcceptors(peptide_molecule)
        num_h_donors = Lipinski.NumHDonors(peptide_molecule)
        

        return num_h_acceptors, num_h_donors
    else:
        print(f"Invalid peptide sequence: {peptide_sequence}")
        return None

# Apply the function to the 'Sequence' column and create new columns for polarity descriptors
#new_pos['Sequence'] = cleaned_df['Sequence']
d7[['NumHAcceptors', 'NumHDonors',]] = cleaned_df['Sequence'].apply(calculate_polarity).apply(pd.Series)

# Display the new DataFrame with polarity descriptors
d7

,NumHAcceptors,NumHDonors
0,14,16
1,14,16
2,21,21
3,17,19
4,17,19
...,...,...
6093,44,49
6094,21,21
6095,50,50
6096,45,51


In [20]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors

# Create a new DataFrame to store the results
df8 = pd.DataFrame()

def calculate_amino_acid_properties(peptide_sequence):
    ng_count = peptide_sequence.count('C(N)N')
    npa_count = peptide_sequence.count('C(N)C')
    nncaa_count = peptide_sequence.count('D') + peptide_sequence.count('E')

    return ng_count, npa_count, nncaa_count
# Apply the function to the 'Sequence' column and create new columns for amino acid properties
df8[['NG', 'NPA', 'NNCAA']] = cleaned_df['Sequence'].apply(calculate_amino_acid_properties).apply(pd.Series)

# Display the DataFrame with amino acid properties
df8

,NG,NPA,NNCAA
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
6093,0,0,1
6094,0,0,1
6095,0,0,1
6096,0,0,3


In [21]:
cleaned_df.reset_index(drop=True, inplace=True)
descriptors_df.reset_index(drop=True, inplace=True)
d4.reset_index(drop=True, inplace=True)
d6.reset_index(drop=True, inplace=True)
d7.reset_index(drop=True, inplace=True)
df8.reset_index(drop=True, inplace=True)

In [22]:
ll = pd.concat([cleaned_df,descriptors_df,d4,d6,d7,df8], axis=1)
ll = ll.fillna(0)
ll

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,FIKHFIHRF,1,../id/peptide_0.pdb,0.895877,-36.344444,69.555556,0.000000,0.000000,44.444444,0.775664,...,90.0,0.42087,0.42087,0.460317,3067.086602,14,16,0,0,0
1,LIKHILHRL,1,../id/peptide_1.pdb,0.924433,-0.277778,105.388889,0.000000,0.000000,0.000000,0.826213,...,81.0,-0.16893,-0.16893,0.703704,2297.411226,14,16,0,0,0
2,LQLLKQLLKLLKQF,1,../id/peptide_2.pdb,0.703186,-36.621429,-9.914286,85.714286,0.000000,0.000000,0.719872,...,122.0,-0.55040,-0.55040,0.726190,3511.063001,21,21,0,0,0
3,IKQLLHFFQRF,1,../id/peptide_3.pdb,0.753307,-32.572727,0.772727,81.818182,0.000000,0.000000,0.642931,...,106.0,-0.99823,-0.99823,0.520548,3515.886152,17,19,0,0,0
4,LKQFVHFIHRF,1,../id/peptide_4.pdb,0.768256,-34.781818,3.081818,81.818182,0.000000,0.000000,0.642681,...,106.0,-0.68803,-0.68803,0.493151,3644.669989,17,19,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0,../id/peptide_6093.pdb,0.640253,-48.727586,-28.479310,93.103448,0.000000,0.000000,1.300180,...,213.0,-17.14109,-17.14109,0.676923,7002.208271,44,49,0,0,1
6094,YNVHKHTDGKC,0,../id/peptide_6094.pdb,0.806652,-17.345455,99.027273,0.000000,0.000000,27.272727,0.736583,...,91.0,-6.66770,-6.66770,0.545455,2919.480801,21,21,0,0,1
6095,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0,../id/peptide_6095.pdb,0.642829,-48.340000,17.716667,56.666667,0.000000,13.333333,1.144406,...,260.0,-5.00823,-5.00823,0.634286,9329.927351,50,50,0,0,1
6096,QTLKLICKECGYTIHREGIRLRKLEIVG,0,../id/peptide_6096.pdb,0.782445,-49.221429,95.907143,0.000000,28.571429,10.714286,1.473323,...,228.0,-10.02109,-10.02109,0.694444,7619.734557,45,51,0,0,3


In [26]:
from Bio import SeqIO

# Save sequences to a FASTA file for CD-HIT
with open("all_sequences.fasta", "w") as f:
    for i, seq in enumerate(ll["Sequence"]):
        f.write(f">seq_{i}\n{seq}\n")

In [27]:
import subprocess

identity = 0.95  # 90% identity
cmd = f"cd-hit -i all_sequences.fasta -o clustered.fasta -c {identity}"

subprocess.run(cmd, shell=True)

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i all_sequences.fasta -o clustered.fasta -c
         0.95

Started: Mon Dec  1 22:32:27 2025
                            Output                              
----------------------------------------------------------------
total seq: 5731
longest and shortest : 39 and 11
Total letters: 129883
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90385568

comparing sequences from          0  to       5731
.....
     5731  finished       5048  clusters

Approximated maximum memory consumption: 77M
writing new database
writing clustering information
program completed !

Total CPU time 0.12


CompletedProcess(args='cd-hit -i all_sequences.fasta -o clustered.fasta -c 0.95', returncode=0)

In [28]:
representative_ids = []

with open("clustered.fasta") as f:
    for line in f:
        if line.startswith(">"):
            rep_id = line.strip().replace(">", "")
            representative_ids.append(rep_id)

In [29]:
kept_indices = [int(x.split("_")[1]) for x in representative_ids]

In [30]:
ll_filtered = ll.iloc[kept_indices].reset_index(drop=True)
ll_filtered

,Sequence,Class,PDB_File,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,LKQFVHFIHRF,1,../id/peptide_4.pdb,0.768256,-34.781818,3.081818,81.818182,0.000000,0.000000,0.642681,...,106.0,-0.68803,-0.68803,0.493151,3644.669989,17,19,0,0,0
1,FHYFWHWFHRF,1,../id/peptide_5.pdb,0.774450,-36.781818,7.309091,81.818182,0.000000,0.000000,0.692914,...,126.0,2.77407,2.77407,0.263736,5645.489133,17,21,0,0,0
2,LYHFLHWFQRF,1,../id/peptide_6.pdb,0.726155,-32.063636,0.245455,81.818182,0.000000,0.000000,0.625450,...,116.0,1.19927,1.19927,0.378049,4587.737874,17,20,0,0,0
3,FLKALFKVASKVL,1,../id/peptide_10.pdb,0.737932,-33.892308,-8.684615,84.615385,0.000000,0.000000,0.695165,...,104.0,-0.42930,-0.42930,0.657534,3051.612827,18,18,0,0,0
4,FLKALFKVALKVL,1,../id/peptide_11.pdb,0.731396,-32.023077,-10.023077,84.615385,0.000000,0.000000,0.690171,...,106.0,1.62450,1.62450,0.671053,3102.804157,17,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5043,GQLANAVGAGIKSMGETLRGFLRQKSAKR,0,../id/peptide_6093.pdb,0.640253,-48.727586,-28.479310,93.103448,0.000000,0.000000,1.300180,...,213.0,-17.14109,-17.14109,0.676923,7002.208271,44,49,0,0,1
5044,YNVHKHTDGKC,0,../id/peptide_6094.pdb,0.806652,-17.345455,99.027273,0.000000,0.000000,27.272727,0.736583,...,91.0,-6.66770,-6.66770,0.545455,2919.480801,21,21,0,0,1
5045,KTLCYSKSEQMLRYSIKLLLYYLKYQIVPI,0,../id/peptide_6095.pdb,0.642829,-48.340000,17.716667,56.666667,0.000000,13.333333,1.144406,...,260.0,-5.00823,-5.00823,0.634286,9329.927351,50,50,0,0,1
5046,QTLKLICKECGYTIHREGIRLRKLEIVG,0,../id/peptide_6096.pdb,0.782445,-49.221429,95.907143,0.000000,28.571429,10.714286,1.473323,...,228.0,-10.02109,-10.02109,0.694444,7619.734557,45,51,0,0,3


In [31]:
categorical_df = ll_filtered.copy()

# Convert the "structural_class" column to categorical
categorical_df['structural_class'] = categorical_df['structural_class'].astype('category')

# Assign categorical codes to the categories
categorical_df['structural_class'] = categorical_df['structural_class'].cat.codes
categorical_df
categorical_df.to_csv('Int-charge-density.csv', index=False)

In [32]:
# Then drop unnecessary columns
X = categorical_df.drop(['Class','Sequence','PDB_File','B','Z','X','O','J','U'], axis=1)
X = X.fillna(0)
y = categorical_df['Class']
y=y.astype('int')

X.columns

Index(['Mean_RSA', 'Mean_Phi', 'Mean_Psi', '%Helix', '%Sheet', '%Coil', 'RoG',
       'SASA', 'Compactness', 'aliphatic_index', 'boman', 'charge',
       'isoelectric_point', 'hydrophobic_moment', 'hydrophobicity',
       'instability_index', 'mass_shift', 'molecular_weight', 'mz',
       'structural_class', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
       'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'TPSA',
       'HeavyAtomCount', 'LogP', 'MolLogP', 'FractionCSP3', 'BertzCT',
       'NumHAcceptors', 'NumHDonors', 'NG', 'NPA', 'NNCAA'],
      dtype='object')

In [33]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume X is your DataFrame
scaler = MinMaxScaler()

# Apply Min-Max Normalization to all columns in X
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display the normalized DataFrame
print("Normalized DataFrame (Min-Max Scaling):")
X_normalized


Normalized DataFrame (Min-Max Scaling):


,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.623324,0.434982,0.189045,0.862408,0.00000,0.000000,0.021204,0.075462,0.355465,0.324242,...,0.119691,0.822653,0.822653,0.402766,0.140064,0.049180,0.097222,0.0,0.0,0.000000
1,0.635614,0.416739,0.210576,0.862408,0.00000,0.000000,0.040316,0.120395,0.424323,0.000000,...,0.196911,0.914488,0.914488,0.000000,0.312772,0.049180,0.125000,0.0,0.0,0.000000
2,0.539786,0.459776,0.174599,0.862408,0.00000,0.000000,0.014649,0.073781,0.478940,0.236364,...,0.158301,0.872715,0.872715,0.200690,0.221468,0.049180,0.111111,0.0,0.0,0.000000
3,0.563153,0.443096,0.129116,0.891892,0.00000,0.000000,0.041172,0.084208,0.302908,0.500000,...,0.111969,0.829516,0.829516,0.691362,0.088872,0.065574,0.083333,0.0,0.0,0.000000
4,0.550185,0.460146,0.122299,0.891892,0.00000,0.000000,0.039272,0.083566,0.327880,0.600000,...,0.119691,0.883994,0.883994,0.715096,0.093290,0.049180,0.069444,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5043,0.369339,0.307775,0.028297,0.981361,0.00000,0.000000,0.271346,0.370973,0.477043,0.258621,...,0.532819,0.386221,0.386221,0.725402,0.429882,0.491803,0.513889,0.0,0.0,0.090909
5044,0.699508,0.594028,0.677718,0.000000,0.00000,0.375000,0.056929,0.047335,0.271123,0.087879,...,0.061776,0.664037,0.664037,0.494592,0.077466,0.114754,0.125000,0.0,0.0,0.090909
5045,0.374450,0.311311,0.263584,0.597297,0.00000,0.183333,0.212083,0.518895,0.469987,0.422222,...,0.714286,0.708056,0.708056,0.650547,0.630808,0.590164,0.527778,0.0,0.0,0.090909
5046,0.651476,0.303271,0.661827,0.000000,0.32967,0.147321,0.337217,0.567987,0.235642,0.405952,...,0.590734,0.575085,0.575085,0.756163,0.483186,0.508197,0.541667,0.0,0.0,0.272727


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [35]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
scaler = MinMaxScaler()

# Apply Min-Max Normalization to all columns in X
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display the normalized DataFrame
print("Normalized DataFrame (Min-Max Scaling):")
X_normalized

Normalized DataFrame (Min-Max Scaling):


,Mean_RSA,Mean_Phi,Mean_Psi,%Helix,%Sheet,%Coil,RoG,SASA,Compactness,aliphatic_index,...,HeavyAtomCount,LogP,MolLogP,FractionCSP3,BertzCT,NumHAcceptors,NumHDonors,NG,NPA,NNCAA
0,0.623324,0.434982,0.189045,0.862408,0.00000,0.000000,0.021204,0.075462,0.355465,0.324242,...,0.119691,0.822653,0.822653,0.402766,0.140064,0.049180,0.097222,0.0,0.0,0.000000
1,0.635614,0.416739,0.210576,0.862408,0.00000,0.000000,0.040316,0.120395,0.424323,0.000000,...,0.196911,0.914488,0.914488,0.000000,0.312772,0.049180,0.125000,0.0,0.0,0.000000
2,0.539786,0.459776,0.174599,0.862408,0.00000,0.000000,0.014649,0.073781,0.478940,0.236364,...,0.158301,0.872715,0.872715,0.200690,0.221468,0.049180,0.111111,0.0,0.0,0.000000
3,0.563153,0.443096,0.129116,0.891892,0.00000,0.000000,0.041172,0.084208,0.302908,0.500000,...,0.111969,0.829516,0.829516,0.691362,0.088872,0.065574,0.083333,0.0,0.0,0.000000
4,0.550185,0.460146,0.122299,0.891892,0.00000,0.000000,0.039272,0.083566,0.327880,0.600000,...,0.119691,0.883994,0.883994,0.715096,0.093290,0.049180,0.069444,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5043,0.369339,0.307775,0.028297,0.981361,0.00000,0.000000,0.271346,0.370973,0.477043,0.258621,...,0.532819,0.386221,0.386221,0.725402,0.429882,0.491803,0.513889,0.0,0.0,0.090909
5044,0.699508,0.594028,0.677718,0.000000,0.00000,0.375000,0.056929,0.047335,0.271123,0.087879,...,0.061776,0.664037,0.664037,0.494592,0.077466,0.114754,0.125000,0.0,0.0,0.090909
5045,0.374450,0.311311,0.263584,0.597297,0.00000,0.183333,0.212083,0.518895,0.469987,0.422222,...,0.714286,0.708056,0.708056,0.650547,0.630808,0.590164,0.527778,0.0,0.0,0.090909
5046,0.651476,0.303271,0.661827,0.000000,0.32967,0.147321,0.337217,0.567987,0.235642,0.405952,...,0.590734,0.575085,0.575085,0.756163,0.483186,0.508197,0.541667,0.0,0.0,0.272727


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [37]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

# Create the XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=60, learning_rate=0.25, max_depth=2, random_state=42)

# Perform 10-fold cross-validation
cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=10)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Fit the model on the entire training set
xgb_classifier.fit(X_train, y_train)

# Predictions on the training set
y_train_pred = xgb_classifier.predict(X_train)

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

# Predictions on the test set
y_test_pred = xgb_classifier.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Calculate precision on the test set
precision = precision_score(y_test, y_test_pred)
print("Precision:", precision)

# Calculate sensitivity (recall) on the test set
recall = recall_score(y_test, y_test_pred)
print("Sensitivity (Recall):", recall)

# Calculate ROC AUC score on the test set
roc_auc = roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:,1])
print("ROC AUC:", roc_auc)





Cross-Validation Scores: [0.88366337 0.88613861 0.9009901  0.90346535 0.9009901  0.8960396
 0.90841584 0.90594059 0.86104218 0.89081886]
Mean CV Accuracy: 0.8937504606540058
Train Accuracy: 0.9170381376919267
Test Accuracy: 0.8930693069306931
Precision: 0.8984881209503239
Sensitivity (Recall): 0.8721174004192872
ROC AUC: 0.9530288191125742
